# Requirements

In [30]:
import json
import re
import pandas as pd
import nltk
pd.options.display.max_columns = 1000 # to see all columns
import warnings
warnings.filterwarnings('ignore')

# Loading datasets

In [31]:
EDHCS = pd.read_csv("../data/large_data/EDHCSg.csv", low_memory=False, index_col=None)
EDHCS.head(5)

,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,language,id,edh_geography_uri,commentary,trismegistos_uri,not_before,external_image_uris,fotos,coordinates,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_word,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,geometry,within_RE,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment
0,Feraudi,epitaph,4 cm,200.0,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,"[{'nomen': 'Sextius', 'praenomen': 'C.', 'pers...",NaN,marble: rocks - metamorphic rocks,tabula,Roma,85 cm,C(aius) Sextius Paris / qui vixit / annis LXX,Italy,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Roma,2014-04-07,Lazio,Roma,Latin,HD000002,https://edh-www.adw.uni-heidelberg.de/edh/geog...,AE 1983: Breite: 35 cm.,https://www.trismegistos.org/text/265631,51.0,{},{},"[12.4823, 41.8955]",265631.0,"['http://www.trismegistos.org/place/000172', '...",Caius Sextius Paris qui vixit annis LXX ...,51 AD – 200 AD,21.0,unbestimmt,257.0,Tafel,48.0,Marmor,1000.0,92.0,Grabinschrift,epitaph,Certain,28.0,85.0,NaN,Marble,tabula,Certain,Roma,Certain,Italy,Certain,Roma,Certain,Lazio,Certain,Roma,Certain,"Via Nomentana, S. Alessandro, Kirche",Certain,51 AD – 200 AD,C Sextius Paris qui vixit annis LXX,Caius Sextius Paris qui vixit annis LXX,Caius Sextius Paris qui vixit annis LXX,"Via Nomentana, S. Alessandro, Kirche",1937,NaN,NaN,NaN,NaN,NaN,"[12.4823, 41.8955]",True,big,True,Roma,992,923313.0,"[12.485098, 41.892777]",big,0.003904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feraudi,honorific inscription,4.5-3 cm,170.0,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,"[{'nomen': 'Mummius+', 'gender': 'male', 'prae...",(12) cm,marble: rocks - metamorphic rocks,statue base,Baetica,(34) cm,[P(ublio) M]ummio [P(ubli) f(ilio)] / [Gal(eri...,Spain,https://edh-www.adw.uni-heidelberg.de/edh/insc...,NaN,2006-08-31,Sevilla,Tomares,Latin,HD000003,https://edh-www.adw.uni-heidelberg.de/edh/geog...,(B): [S]isenna ist falscher Kasus; folgende E...,https://www.trismegistos.org/text/220675,131.0,{},{},"[-6.0459, 37.3728]",220675.0,"['http://www.trismegistos.org/place/025443', '...",Publio Mummio Publi filio Galeria Sisennae Rut...,131 AD – 170 AD,21.0,unbestimmt,57.0,Statuenbasis,48.0,Marmor,1000.0,69.0,Ehreninschrift,honorific inscription,Certain,37.0,34.0,12.0,Marble,statue base,Certain,Baetica,Certain,Spain,Certain,NaN,NaN,Sevilla,Certain,Tomares,Certain,NaN,NaN,131 AD – 170 AD,ummio isenna Xv,Publio Mummio Publi filio Galeria Sisennae Rut...,Publio Mummio Publi filio Galeria Sisennae Rut...,NaN,before 19

In [32]:
print(EDHCS.columns)

Index(['responsible_individual', 'type_of_inscription', 'letter_size',
       'not_after', 'literature', 'work_status', 'height', 'diplomatic_text',
       'people', 'depth',
       ...
       'inscription_stripped_final', 'Links', 'dating from', 'dating to',
       'status', 'Latitude', 'Longitude', 'photo', 'Material', 'Comment'],
      dtype='object', length=109)


# Function to extract occupations

In [33]:
def extract_occup(inscription_text):
    occups_found = []
    if not isinstance(inscription_text, str): # if not valid string
        inscription_text = ""
    for occup in occups_declined_dict.keys():
        for occup_morph in occups_declined_dict[occup]:
            try:
                if occup_morph in inscription_text: # first check it this way, otherwise skip
                    occup_morph_N = len(re.findall("(\W|^)" + occup_morph + "(\W|$)", inscription_text))
                    occups_found.extend([occup] * occup_morph_N)
                    inscription_text = inscription_text.replace(occup_morph, "")
            except: pass
    return occups_found

# Occupations - extraction

In [34]:
# read declined occupations
with open("../data/occups_declined_dict.json", "r") as fp:
    occups_declined_dict = json.load(fp)

In [35]:
# check that our occupations are properly arranged (from the longest...)
list(occups_declined_dict.keys())[:20]

['exactor auri argenti et aeris',
 'inclusor auri et gemmarum',
 'tesserarius lignarius',
 'refector pectinarius',
 'instructor parietum',
 'tribor argentarius',
 'carrocarpentarius',
 'dactilidiogliphus',
 'manceps salinarum',
 'manceps thermarum',
 'aequator monetae',
 'conchyliolegulus',
 'orchestopolarius',
 'rhypararographus',
 'calciamentarius',
 'chorographarius',
 'diffusor oleari',
 'domnicomontanus',
 'faber tignuarii',
 'medicamentarius']

In [36]:
%%time
EDHCS["occups"] = EDHCS["clean_text_interpretive_word"].apply(extract_occup)

CPU times: user 1min 8s, sys: 3.75 ms, total: 1min 8s
Wall time: 1min 8s


In [37]:
EDHCS["occups_N"] = EDHCS["occups"].apply(len)

In [38]:
EDHCS["occups_N"].sum()

3886

In [39]:
EDHCS_occups_list = [el for sublist in EDHCS["occups"].tolist() for el in sublist]
occupations_counts = pd.DataFrame(nltk.FreqDist(EDHCS_occups_list).most_common(), columns=["occupation", "count"])
occupations_counts.head(10)

,occupation,count
0,faber,658
1,curator,389
2,medicus,248
3,aerarius,227
4,scriba,211
5,sagittarius,128
6,vexillarius,75
7,mensor,74
8,negotiator,71
9,dendrophorus,68


In [40]:
# load the occupation list
occupations_df = pd.read_csv("../data/occupations_list.csv")
occupation_category_dict = dict(zip(occupations_df["Term"], occupations_df["Category"]))
occupation_translation_dict = dict(zip(occupations_df["Term"], occupations_df["Translation_eng"]))
# add category and translation to the counts
occupations_counts["category"] = occupations_counts["occupation"].apply(lambda x: occupation_category_dict[x])
occupations_counts["translation"] = occupations_counts["occupation"].apply(lambda x: occupation_translation_dict[x])
occupations_counts.head(10)

,occupation,count,category,translation
0,faber,658,Metal-Working,"a worker in wood, stone, metal, etc., a forger..."
1,curator,389,Managerial,"he who takes charge, a manager, overseer, supe..."
2,medicus,248,Miscellaneous Services,a surgeon
3,aerarius,227,Metal-Working,metal worker
4,scriba,211,Education,"a public writer, official scribe, professional..."
5,sagittarius,128,Metal-Working,"arrow-makers, arrow-smiths"
6,vexillarius,75,Miscellaneous Services,"a standard-bearer, maker"
7,mensor,74,Miscellaneous Services,"a measurer, surveyor"
8,negotiator,71,Retail,"one who does business by wholesale, a wholesal..."
9,dendrophorus,68,Building,a carpenter


In [41]:
occupations_counts.to_csv("../data/occupations_counts.csv")

## Occupations - overview

In [42]:
# EDH
EDHCS[EDHCS["id"].notnull()]["occups_N"].sum()

1265

In [43]:
# EDCS
EDHCS[EDHCS["id"].isnull()]["occups_N"].sum()

2621

In [44]:
print("EDHCS - number of occupation occurances: " + str(EDHCS["occups_N"].sum()))
print("EDHCS - number of inscriptions with at least one occupation mentioned: " + str(len(EDHCS[EDHCS["occups_N"] > 0])))

EDHCS - number of occupation occurances: 3886
EDHCS - number of inscriptions with at least one occupation mentioned: 3256


# Organizations - extraction

In [45]:
# read declined occupations
with open("../data/organizations_declined_dict.json", "r") as fp:
    orgs_declined_dict = json.load(fp)

In [46]:
# check that our occupations are properly arranged (from the longest...)
list(orgs_declined_dict.keys())[:15]

['collegatarius',
 'collegiarius',
 'collegiatus',
 'corporatus',
 'sodalicium',
 'collegium',
 'collegius',
 'concilium',
 'conlegium',
 'sodalitas',
 'colegium',
 'sociatio',
 'societas',
 'collega',
 'corpus']

In [47]:
def extract_orgs(inscription_text):
    orgs_found = []
    if not isinstance(inscription_text, str): # if not valid string
        inscription_text = ""
    for org in orgs_declined_dict.keys():
        for orgs_morph in orgs_declined_dict[org]:
            try:
                if orgs_morph in inscription_text: # first check it this way, otherwise skip
                    orgs_morph_N = len(re.findall("(\W|^)" + orgs_morph + "(\W|$)", inscription_text))
                    orgs_found.extend([org] * orgs_morph_N)
                    inscription_text = inscription_text.replace(org_morph, "")
            except: pass
    return orgs_found

In [48]:
%%time
EDHCS["organizations"] = EDHCS["clean_text_interpretive_word"].apply(extract_orgs)

CPU times: user 1.68 s, sys: 0 ns, total: 1.68 s
Wall time: 1.68 s


In [49]:
EDHCS["organizations_N"] = EDHCS["organizations"].apply(len)

In [50]:
EDHCS_organizations_list = [el for sublist in EDHCS["organizations"].tolist() for el in sublist]
print(len(EDHCS_organizations_list))
print(nltk.FreqDist(EDHCS_organizations_list).most_common(30))

2923
[('collegius', 1155), ('collegium', 1145), ('corpus', 380), ('collega', 110), ('colegium', 48), ('sodalicium', 17), ('corporatus', 16), ('collegiatus', 14), ('conlegium', 13), ('concilium', 12), ('societas', 12), ('collegiarius', 1)]


In [51]:
organizations_counts = pd.DataFrame(nltk.FreqDist(EDHCS_organizations_list).most_common(), columns=["term", "count"])
organizations_counts

,term,count
0,collegius,1155
1,collegium,1145
2,corpus,380
3,collega,110
4,colegium,48
5,sodalicium,17
6,corporatus,16
7,collegiatus,14
8,conlegium,13
9,concilium,12


In [52]:
# EDH
EDHCS[EDHCS["id"].notnull()]["organizations_N"].sum()

907

In [53]:
# EDCS
EDHCS[EDHCS["id"].isnull()]["organizations_N"].sum()

2016

In [54]:
organizations_counts.to_csv("../data/organizations_counts.csv")

## Organizations - overview

In [55]:
print("EDHCS - number of organization occurances: " + str(EDHCS["organizations_N"].sum()))
print("EDHCS - number of inscriptions with at least one organization mentioned: " + str(len(EDHCS[EDHCS["organizations_N"] > 0])))

EDHCS - number of organization occurances: 2923
EDHCS - number of inscriptions with at least one organization mentioned: 1229


In [56]:
EDHCS.to_csv("../data/large_data/EDHCS_occupsorgs.csv", index=False)

In [67]:
#import sddk 
#conf = sddk.configure("SDAM_root", "648597@au.dk")
#sddk.write_file("SDAM_data/social_diversity/EDHCS_occupsorgs_2021-03-24.json", EDHCS, conf)

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/social_diversity/EDHCS_occupsorgs_2021-03-24.json"
